# Modeling Test #1
### 사용자 입력값 + VOD 줄거리

<br><hr>

## 00. 기본 설정

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import google.generativeai as genai
import typing_extensions as typing
from dotenv import load_dotenv
import os

import json

# 경고 메시지 출력 X
import warnings
warnings.filterwarnings("ignore")

In [3]:
# 한글 font 설정
import platform
import matplotlib.font_manager as fm

#matplotlib 패키지 한글 깨짐 처리 시작
#------------------------------------------------------------------------------------
# 운영체제별 한글 폰트 설정

if platform.system() == 'Darwin': # Mac 환경 폰트 설정
    plt.rc('font', family='AppleGothic')
elif platform.system() == 'Windows': # Windows 환경 폰트 설정
    plt.rc('font', family='Malgun Gothic')
    
plt.rcParams['axes.unicode_minus'] = False #한글 폰트 사용시 마이너스 폰트 깨짐 해결

In [4]:
# 글씨 선명하게 출력하는 설정

from IPython.display import set_matplotlib_formats
set_matplotlib_formats("retina")

<br><br><hr>

## 01. Gemini API 환경설정 및 초기화

In [1]:
# .env 파일에서 Gemini API Key 가져오기
load_dotenv()
Gemini_API_KEY = os.environ.get('API_KEY_GEMINI')

NameError: name 'load_dotenv' is not defined

In [6]:
# gemini API key 및 model 설정
genai.configure(api_key=Gemini_API_KEY)
model = genai.GenerativeModel('gemini-1.5-flash')

<br><br><hr>

## 02. 사용자 입력 키워드 추출 w/ Gemini

In [7]:
# 응답으로 받을 JSON 형식 정의
class Keyword(typing.TypedDict):
  user_id: int                    # 사용자를 구분하기 위한 셋톱박스ID
  keywords: list[str]             # 키워드를 리스트에 저장 

In [8]:
# 사용자 입력 텍스트에서 키워드를 추출하는 함수 정의
def extract_keywords(user_input, user_id):

  # model.generate_content 메서드로 Gemini에게 사용자 입력 텍스트 전달 
  response = model.generate_content(
    # 전달된 prompt에는 사용자ID와 사용자의 채팅이 포함됨
    f"""
    사용자ID: {user_id}
    문장: "{user_input}"
    위의 입력에서 주요 키워드를 추출하여 다음 JSON 스키마에 맞게 반환하세요:
    """,
    
    generation_config=genai.GenerationConfig(   # 응답 형식 지정
      response_mime_type="application/json",    # JSON 형식으로 응답을 받기 위한 설정
      response_schema=Keyword                   # 응답이 위에서 정의한 Keyword의 구조를 따르도록 함
    ),
  )

  # 결과 텍스트 파싱
  try:
    # 반환된 텍스트에서 JSON 데이터 추출
    raw_response = response.candidates[0].content.parts[0].text

    # 텍스트는 JSON 형식으로 되어있으므로 json.loads를 사용하여 dict. 자료형으로 변환하여 parsed_response 변수에 저장
    parsed_response = json.loads(raw_response)

    # user_id와 keywords 추출하여 변수에 저장
    user_id = parsed_response.get("user_id", None)
    keywords = parsed_response.get("keywords", [])

    # 결과를 형식화하여 반환
    result_text = f"{user_id}: {keywords}"
    return result_text
  
  # 예외처리
  except Exception as e:
    print(f"Error parsing response: {e}")
    return None

In [10]:
# 사용자 입력값
user_input = "킬링타임용 영화를 추천해줘"
# extract_keywords함수 호출
keywords = extract_keywords(user_input, user_id=123456)

print("사용자 입력:", user_input)
print(keywords)

사용자 입력: 킬링타임용 영화를 추천해줘
123456: ['킬링타임', '영화', '추천']


<br><br><hr>

## 03. 영화 데이터 로드

In [11]:
movies = pd.read_csv('..\data\movies_4000_tmdb_genre.csv')
print(movies.shape)   # (4241, 6)
movies.head(3)

(4241, 6)


,asset_nm_전처리,ct_cl,genre_of_ct_cl,summary_최신순,최신순,genre_tmdb
0,귀멸의 칼날: 남매의 연,영화,애니메이션,혈귀의 습격으로 가족을 잃은 소년 ‘탄지로’. 유일하게 살아남은 여동생 ‘네즈코’마...,2019-03-29,"Animation, Action, Fantasy, Thriller"
1,색에 놀다,영화,에로틱,하얀 색의 순수하고 착한 사랑을 꿈꾸는 25살 모태 솔로 지수. 그녀의 짝사랑 상대...,2017-01-01,"Thriller, Drama, Romance"
2,돌이킬 수 없는 주말,영화,공포/스릴러,베키는 결혼을 앞두고 친구 수잔과 함께 다트무어로 여행을 떠난다. 그곳에서 신비한 ...,2015-09-18,"Drama, Horror, Mystery"


In [12]:
movies['genre_of_ct_cl'].value_counts()

genre_of_ct_cl
드라마        1011
액션/어드벤쳐     977
공포/스릴러      639
성인          340
멜로          329
코미디         265
애니메이션       169
SF/환타지      159
다큐멘터리       126
기타           88
무협           37
로맨틱코미디       37
에로틱          35
단편           12
서부            7
뮤지컬           6
역사            2
인물            2
Name: count, dtype: int64

##### *>> 장르가 '성인' 또는 '기타'인 행와 장르의 value_counts가 40개 미만인 행은 삭제*

In [13]:
# 각 장르의 수 를 genre_counts 변수에 저장
genre_counts = movies['genre_of_ct_cl'].value_counts()

In [14]:
# 삭제할 조건 생성
delete_conditions = (movies['genre_of_ct_cl'] == '성인') | (movies['genre_of_ct_cl'] == '기타') | (movies['genre_of_ct_cl'].isin(genre_counts[genre_counts < 40].index))
filtered_movies = movies[~delete_conditions]

In [15]:
# 결과 확인
print(filtered_movies.shape)
filtered_movies['genre_of_ct_cl'].value_counts()

### (4241, 6) >> (3675, 6) >> 566개의 행 삭제

(3675, 6)


genre_of_ct_cl
드라마        1011
액션/어드벤쳐     977
공포/스릴러      639
멜로          329
코미디         265
애니메이션       169
SF/환타지      159
다큐멘터리       126
Name: count, dtype: int64

##### *>> 사용하지 않을 열 drop*

In [16]:
col_to_drop = ['ct_cl', '최신순', 'genre_tmdb']
drop_movies = filtered_movies.drop(columns=col_to_drop, axis=1)

drop_movies.head(3)

,asset_nm_전처리,genre_of_ct_cl,summary_최신순
0,귀멸의 칼날: 남매의 연,애니메이션,혈귀의 습격으로 가족을 잃은 소년 ‘탄지로’. 유일하게 살아남은 여동생 ‘네즈코’마...
2,돌이킬 수 없는 주말,공포/스릴러,베키는 결혼을 앞두고 친구 수잔과 함께 다트무어로 여행을 떠난다. 그곳에서 신비한 ...
3,섹스 앤 머니,액션/어드벤쳐,갱단 두목 페페는 라이벌 갱단 두목 조조와 세력 다툼을 벌이다 쫓기는 신세가 된다....


<br><br><hr>

## 04. 영화 줄거리 벡터화

##### *a. 텍스트 임베딩 생성 (w/ SBERT | OpenAI Embedding API)*

In [17]:
import faiss
from sentence_transformers import SentenceTransformer

In [33]:
# 1. 텍스트를 임베딩(숫자 벡터)로 변환하기 위한 모델 로드 (Sentence-BERT)
# embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# 임베딩 모델 출처: https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
embedding_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [34]:
# 2. 영화 데이터 줄거리 ('summary_최신순') 벡터화

# 각 영화의 줄거리를 리스트로 가져와서 >> 각 줄거리를 고정된 크기의 벡터로 변환하여 movie_embeddings 변수에 저장 
movie_embeddings = embedding_model.encode(drop_movies['summary_최신순'].tolist(), show_progress_bar=True)

# FAISS에서 처리 가능하도록 벡터를 float32 형식으로 변환
movie_embeddings = np.array(movie_embeddings).astype('float32')
movie_embeddings

Batches:   0%|          | 0/115 [00:00<?, ?it/s]

array([[-0.01222587,  0.26877487, -0.03638799, ..., -0.02020498,
         0.08367279, -0.07963274],
       [ 0.06627268, -0.10923017, -0.00191216, ..., -0.04519046,
        -0.13687882, -0.10875412],
       [-0.00652586,  0.14733294,  0.00066011, ...,  0.06406496,
         0.01523128,  0.05121128],
       ...,
       [ 0.05330199,  0.06797754, -0.48861244, ...,  0.07119494,
        -0.13342021,  0.14347412],
       [ 0.04077281,  0.17732199, -0.14524025, ..., -0.12641034,
         0.17938673,  0.07783416],
       [ 0.19016382,  0.144844  ,  0.02407256, ...,  0.09320447,
        -0.14046723,  0.16669984]], shape=(3675, 384), dtype=float32)

In [36]:
# 3. Faiss로 벡터 색인화 (정렬)
dimension = movie_embeddings.shape[1]        # 임베딩 벡터의 차원 수 확인
print(dimension)                             # >> 384차원

384


In [40]:
# 4-1. 코사인 거리 기반 유사도 측정
index_cos = faiss.IndexFlatIP(dimension)
faiss.normalize_L2(movie_embeddings)         # 인덱스를 생성하기 전 정규화
index_cos.add(movie_embeddings)              # 정규화를 한 후 add
print(f"FAISS Index에 {index_cos.ntotal}개의 영화 벡터 추가")

FAISS Index에 3675개의 영화 벡터 추가


In [39]:
# 4-2. 유클리디안 거리 기반 유사도 측정
index_l2 = faiss.IndexFlatL2(dimension)      # 얘는 굳이 정규화하지 않고
index_l2.add(movie_embeddings)               # 바로 add
print(f"FAISS Index에 {index_l2.ntotal}개의 영화 벡터 추가")

# 유클리디안 거리를 구할 때 정규화를 하지 않는 이유는?
# >> 벡터 크기를 포함하고 있기 떄문

FAISS Index에 3675개의 영화 벡터 추가


##### *b. FAISS로 유사도 검색*

In [66]:
# 5-1. 키워드 기반 추천 함수 (코사인)
def recommend_vods_cos(user_input, k):
  # keywords = extract_keywords(user_input, user_id=123456)
  # print("키워드: ", keywords)

  # if not keywords:
  #   print("키워드 추출 실패 흑흑")
  #   return []
  
  # # 키워드를 문장으로 조합
  # keyword_to_sentence = " ".join(keywords)

  # 조합한 키워드 문장 벡터화 + 정규화
  user_input_embedding = embedding_model.encode([user_input])
  user_input_embedding = np.array(user_input_embedding).astype('float32')
  faiss.normalize_L2(user_input_embedding)

  # FAISS 코사인 유사도 검색
  distance, indices = index_cos.search(user_input_embedding, k)

  # 결과 출력
  recommendations = []
  for idx, dist in zip(indices[0], distance[0]):
    recommendations.append({
      "제목": movies.iloc[idx]['asset_nm_전처리'],
      "장르": movies.iloc[idx]['genre_of_ct_cl'],
      "줄거리": movies.iloc[idx]['summary_최신순'],
      "코사인 유사도": dist
    })
  return recommendations

In [67]:
# 5-2. 키워드 기반 추천 함수 (유클리디안)
def recommend_vods_l2(user_input, k):
  # keywords = extract_keywords(user_input, user_id=123456)
  # print("키워드: ", keywords)

  # if not keywords:
  #   print("키워드 추출 실패 흑흑")
  #   return []
  
  # 키워드를 문장으로 조합
  # keyword_to_sentence = " ".join(keywords)

  # 조합한 키워드 문장 벡터화 + 정규화
  user_input_embedding = embedding_model.encode([user_input])
  user_input_embedding = np.array(user_input_embedding).astype('float32')

  # FAISS 유클리디안 거리 검색
  distance, indices = index_l2.search(user_input_embedding, k)

  # 결과 출력
  recommendations = []
  for idx, dist in zip(indices[0], distance[0]):
    recommendations.append({
      "제목": movies.iloc[idx]['asset_nm_전처리'],
      "장르": movies.iloc[idx]['genre_of_ct_cl'],
      "줄거리": movies.iloc[idx]['summary_최신순'],
      "유클리디안 거리": dist
    })
  return recommendations

##### *c. 유사도가 가장 높은 N개의 영화 추천*

In [71]:
# 6-1. 사용자 입력 (코사인 유사도)
user_input = "소녀 버니스와 친구들의 탈출 시도"
recommendations_cos = recommend_vods_cos(user_input, 5)

# 코사인 유사도 추천 결과 출력
print("\n코사인 유사도 기반으로 추천된 콘텐츠:")
for rec in recommendations_cos:
    print(f"""{rec['제목']} - {rec['장르']}, \t\t 코사인 유사도: {rec['코사인 유사도']:.4f}")""")


print("\n--------------------------------------\n")
# 6-2. 사용자 입력 (유클리디안 거리 유사도)
recommendations_l2 = recommend_vods_l2(user_input, 5)

# 유클리디안 거리 추천 결과 출력
print("\n유클리디안 거리 기반으로 추천된 콘텐츠:")
for rec in recommendations_l2:
    print(f"""{rec['제목']} - {rec['장르']} \t\t 유클리디안 거리: {rec['유클리디안 거리']:.4f}")""")


코사인 유사도 기반으로 추천된 콘텐츠:
쁘띠 마망 - 드라마, 		 코사인 유사도: 0.6906")
타워 - 드라마, 		 코사인 유사도: 0.6744")
꽉 물어주는 여자 - 성인, 		 코사인 유사도: 0.6236")
앤트맨과 와스프 - 액션/어드벤쳐, 		 코사인 유사도: 0.6236")
천로역정: 천국을 찾아서 - 애니메이션, 		 코사인 유사도: 0.6067")

--------------------------------------


유클리디안 거리 기반으로 추천된 콘텐츠:
쁘띠 마망 - 드라마 		 유클리디안 거리: 7.0853")
타워 - 드라마 		 유클리디안 거리: 7.1909")
앤트맨과 와스프 - 액션/어드벤쳐 		 유클리디안 거리: 7.5214")
꽉 물어주는 여자 - 성인 		 유클리디안 거리: 7.5214")
천로역정: 천국을 찾아서 - 애니메이션 		 유클리디안 거리: 7.6313")
